# 🧭 Funnel Drop-Off Analysis (2024)

This notebook:
- Loads `data/events.csv`
- Builds funnel metrics (overall + by device/region)
- Calculates step conversion + drop-off
- Exports CSV outputs for dashboard use


In [1]:
import pandas as pd
import numpy as np
events_path = "data/events.csv"
df = pd.read_csv(events_path, parse_dates=['event_time'])
df.head()

,user_id,event_name,event_time,device,region
0,31294,visit,2024-01-01 00:00:08,mobile,NaN
1,19931,visit,2024-01-01 00:00:20,web,EU
2,38693,visit,2024-01-01 00:00:46,web,NaN
3,45921,visit,2024-01-01 00:00:49,mobile,EU
4,62898,visit,2024-01-01 00:01:05,mobile,NaN


## 1) Define the funnel order

In [2]:
funnel_steps = ['visit','signup','product_view','add_to_cart','purchase']
step_order = {s:i+1 for i,s in enumerate(funnel_steps)}

# keep only funnel events
df = df[df['event_name'].isin(funnel_steps)].copy()
df['step_order'] = df['event_name'].map(step_order)
df['device'] = df['device'].fillna('unknown')
df['region'] = df['region'].fillna('unknown')

df.shape

(126740, 6)

## 2) Earliest time each user hit each step (per segment)
We avoid double-counting repeated events by taking the first time per user-step.

In [3]:
user_step = (
    df.groupby(['user_id','device','region','step_order','event_name'], as_index=False)
      .agg(first_step_time=('event_time','min'))
)
user_step.head()

,user_id,device,region,step_order,event_name,first_step_time
0,1,mobile,APAC,1,visit,2024-04-16 00:18:15
1,2,mobile,unknown,1,visit,2024-11-26 00:24:42
2,3,mobile,EU,1,visit,2024-05-29 00:56:20
3,4,web,unknown,1,visit,2024-07-10 00:15:55
4,4,web,unknown,2,signup,2024-07-10 12:00:00


## 3) Enforce funnel order
A user counts for step N only if they completed step N-1 earlier (same device+region segment).

In [4]:
user_step = user_step.sort_values(['user_id','device','region','step_order'])

# pivot to wide for order checking
wide = user_step.pivot_table(
    index=['user_id','device','region'],
    columns='step_order',
    values='first_step_time',
    aggfunc='min'
).reset_index()

# determine if each step is qualified
qualified = wide[['user_id','device','region']].copy()
qualified[1] = wide[1].notna()  # visit
qualified[2] = wide[2].notna() & (wide[2] >= wide[1])
qualified[3] = wide[3].notna() & qualified[2] & (wide[3] >= wide[2])
qualified[4] = wide[4].notna() & qualified[3] & (wide[4] >= wide[3])
qualified[5] = wide[5].notna() & qualified[4] & (wide[5] >= wide[4])

qualified.head()

step_order,user_id,device,region,1,2,3,4,5
0,1,mobile,APAC,True,False,False,False,False
1,2,mobile,unknown,True,False,False,False,False
2,3,mobile,EU,True,False,False,False,False
3,4,web,unknown,True,True,True,True,False
4,5,mobile,unknown,True,True,True,False,False


## 4) Funnel metrics by device + region

In [5]:
rows = []
for (device, region), g in qualified.groupby(['device','region']):
    users_at_step = {s: int(g[step_order[s]].sum()) for s in funnel_steps}
    prev = None
    for s in funnel_steps:
        u = users_at_step[s]
        if prev is None:
            conv = np.nan
            drop = np.nan
            prev_users = np.nan
        else:
            prev_users = prev
            conv = (u / prev_users) if prev_users else np.nan
            drop = ((prev_users - u) / prev_users) if prev_users else np.nan
        rows.append([device, region, s, step_order[s], u, prev_users, conv, drop])
        prev = u

funnel_metrics = pd.DataFrame(rows, columns=[
    'device','region','step','step_order','users','prev_users','step_conversion_rate','step_dropoff_rate'
])

funnel_metrics['step_conversion_pct'] = (funnel_metrics['step_conversion_rate']*100).round(2)
funnel_metrics['step_dropoff_pct'] = (funnel_metrics['step_dropoff_rate']*100).round(2)
funnel_metrics.sort_values(['device','region','step_order']).head(15)

,device,region,step,step_order,users,prev_users,step_conversion_rate,step_dropoff_rate,step_conversion_pct,step_dropoff_pct
0,mobile,APAC,visit,1,12235,NaN,NaN,NaN,NaN,NaN
1,mobile,APAC,signup,2,4316,12235.0,0.352758,0.647242,35.28,64.72
2,mobile,APAC,product_view,3,3037,4316.0,0.703661,0.296339,70.37,29.63
3,mobile,APAC,add_to_cart,4,678,3037.0,0.223247,0.776753,22.32,77.68
4,mobile,APAC,purchase,5,373,678.0,0.550147,0.449853,55.01,44.99
5,mobile,EU,visit,1,12152,NaN,NaN,NaN,NaN,NaN
6,mobile,EU,signup,2,4238,12152.0,0.348749,0.651251,34.87,65.13
7,mobile,EU,product_view,3,2971,4238.0,0.701038,0.298962,70.10,29.90
8,mobile,EU,add_to_cart,4,730,2971.0,0.245709,0.754291,24.57,75.43
9,mobile,EU,purchase,5,413,730.0,0.565753,0.434247,56.58,43.42


## 5) Overall funnel (all users)

In [6]:
overall = qualified.copy()
overall['device'] = 'all'
overall['region'] = 'all'

rows = []
users_at_step = {s: int(overall[step_order[s]].sum()) for s in funnel_steps}
prev = None
for s in funnel_steps:
    u = users_at_step[s]
    if prev is None:
        conv = np.nan; drop = np.nan; prev_users = np.nan
    else:
        prev_users = prev
        conv = (u / prev_users) if prev_users else np.nan
        drop = ((prev_users - u) / prev_users) if prev_users else np.nan
    rows.append(['all','all',s,step_order[s],u,prev_users,conv,drop])
    prev = u

overall_metrics = pd.DataFrame(rows, columns=funnel_metrics.columns[:8])
overall_metrics['step_conversion_pct'] = (overall_metrics['step_conversion_rate']*100).round(2)
overall_metrics['step_dropoff_pct'] = (overall_metrics['step_dropoff_rate']*100).round(2)
overall_metrics

,device,region,step,step_order,users,prev_users,step_conversion_rate,step_dropoff_rate,step_conversion_pct,step_dropoff_pct
0,all,all,visit,1,75000,NaN,NaN,NaN,NaN,NaN
1,all,all,signup,2,26212,75000.0,0.349493,0.650507,34.95,65.05
2,all,all,product_view,3,18338,26212.0,0.699603,0.300397,69.96,30.04
3,all,all,add_to_cart,4,4648,18338.0,0.253463,0.746537,25.35,74.65
4,all,all,purchase,5,2542,4648.0,0.546902,0.453098,54.69,45.31


## 6) Export outputs for dashboard
These CSVs will be used by Streamlit / Power BI / Tableau.

In [7]:
out1 = 'data/funnel_metrics_by_device_region.csv'
out2 = 'data/funnel_metrics_overall.csv'

funnel_metrics.sort_values(['device','region','step_order']).to_csv(out1, index=False)
overall_metrics.sort_values(['step_order']).to_csv(out2, index=False)

print('Saved:', out1)
print('Saved:', out2)

Saved: data/funnel_metrics_by_device_region.csv
Saved: data/funnel_metrics_overall.csv
